In [ ]:
"""
Use LDA and pyLDAvis for topic modeling visualization

"""

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from collections import Counter

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from wordcloud import WordCloud

from gensim import matutils, models
import scipy.sparse

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/katiehuang/Desktop/metis/projects/onl_ds5_project_4/py')
from word_cloud import *
import importlib

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Let's read in our document-term matrix
speech_df = pd.read_pickle('../dump/speech_clean_lemma')
data = pd.read_pickle('../dump/data_dtm_lemma.pkl')
tdm = data.transpose()
tdm.shape

(36156, 441)

# Visualization with pyLDAvis

In [3]:
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# # from sklearn.datasets import fetch_20newsgroups
# # from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.decomposition import LatentDirichletAllocation


In [4]:
# newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
# docs_raw = newsgroups.data
# print(len(docs_raw))

### Define function

In [5]:
def generate_pyLDAvis(df,column_name,n_components):
    """Input: df and column of interesnt (e.g. transcript, nouns)
       Output: pyLDAvis graph"""
    
    transcripts = df[column_name].tolist()
    docs_raw = transcripts
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    stop_words = 'english',
                                    lowercase = True,
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    max_df = 0.5, 
                                    min_df = 10)
    dtm_tf = tf_vectorizer.fit_transform(docs_raw)
    n_components = n_components
    # for TF DTM
    lda_tf = LatentDirichletAllocation(n_components=n_components, random_state=0)
    lda_tf.fit(dtm_tf)
    
    return pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [6]:
generate_pyLDAvis(speech_df,'transcript',5)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.055707 -0.017697       1        1  29.307788
4      0.130733 -0.002960       2        1  26.584507
3     -0.094807 -0.029086       3        1  22.950453
2      0.029518 -0.073164       4        1  11.974882
0     -0.009737  0.122907       5        1   9.182370, topic_info=            Term        Freq       Total Category  logprob  loglift
1036       dream  717.000000  717.000000  Default  30.0000  30.0000
1302        fear  467.000000  467.000000  Default  29.0000  29.0000
3462  university  563.000000  563.000000  Default  28.0000  28.0000
3626      wonder  284.000000  284.000000  Default  27.0000  27.0000
3623       women  669.000000  669.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
198          art   44.997187  223.316408   Topic5  -6.0814   0.7859
2175     morning   47.722345  293.576423   Topic5  -6.0226   0.5711
51      actually   53.465628  501.034082   Topic5  -5.9089   0.1502
508    challenge   50.989591  455.429637   Topic5  -5.9564   0.1983
2517       power   49.017691  374.865018   Topic5  -5.9958   0.3535

[409 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3         1  0.138820   able
3         2  0.226851   able
3         3  0.291182   able
3         4  0.172678   able
3         5  0.172678   able
...     ...       ...    ...
3678      1  0.311843  youve
3678      2  0.031821  youve
3678      3  0.519739  youve
3678      4  0.067884  youve
3678      5  0.067884  youve

[1227 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])

### Test with Tf-idf
(which looks bad for LDA model)

In [7]:
transcripts = speech_df.transcript.tolist()
docs_raw = transcripts
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(441, 3682)


In [8]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(441, 3682)


In [9]:
n_components = 5
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=n_components, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=n_components, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=5, random_state=0)

In [10]:
# pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [11]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
2      0.042038  0.0       1        1  94.601825
4     -0.010509 -0.0       2        1   1.349544
0     -0.010509  0.0       3        1   1.349544
1     -0.010509  0.0       4        1   1.349544
3     -0.010509  0.0       5        1   1.349544, topic_info=          Term      Freq     Total Category  logprob  loglift
468     career  7.000000  7.000000  Default  30.0000  30.0000
51    actually  7.000000  7.000000  Default  29.0000  29.0000
3156     story  7.000000  7.000000  Default  28.0000  28.0000
3672     youll  7.000000  7.000000  Default  27.0000  27.0000
3132     state  7.000000  7.000000  Default  26.0000  26.0000
...        ...       ...       ...      ...      ...      ...
2258      nose  0.022231  1.140829   Topic5  -8.2111   0.3674
989   distinct  0.022231  0.967292   Topic5  -8.2111   0.5324
1858     itthe  0.022231  0.766763   Topic5  -8.2111   0.7647
3676  youthful  0.022231  0.996610   Topic5  -8.2111   0.5025
581      cling  0.022231  0.715238   Topic5  -8.2111   0.8343

[300 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
11        1  1.674506    absurd
31        1  1.685302  accurate
51        1  0.942097  actually
60        1  1.675966  adequate
79        1  1.040167    advice
...     ...       ...       ...
3535      1  1.033264      walk
3623      1  1.010239     women
3672      1  1.053206     youll
3676      1  1.003401  youthful
3678      1  0.953476     youve

[79 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 2, 4])

In [12]:
# Wow. Indeed Tf-idf is not good for LDA!

## 2. Refine topic modeling
Create gensim corpus and dictionarry

In [13]:
import pickle

with open("../dump/common_words.txt", "rb") as f:   # Unpickling
    common_words = pickle.load(f)

In [14]:
# Bag of words with CountVectorizer
# add_stop_words selected from after lemmatization
# will also remove common_words (most commonly used words in all speeches)
# will also remove boring words (words that do not add much insight to topic modeling)
add_stop_words = ['like','youre','ive','im','really','id','ve','just','dont','thi','wa',
                  'say','know','make','people']

boring_words = ['say','like','just','dont','don','im',
                  'ive','youll','youve','things','thing','youre','right','really','lot',
                  'make','know','people','way','day','class']


add_stop_words = add_stop_words + common_words + boring_words

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(speech_df.transcript)

In [15]:
# If add_stop_words is modified, update tdm
data_dtm = cv_dtm(speech_df,'transcript',add_stop_words)
tdm = data_dtm.transpose()

In [16]:
data_dtm.shape

(441, 36121)

In [17]:
# We're going to put the term-document matrix into a new gensim format
# From df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [18]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
# {dictionsry of location: word}
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
len(id2word)

36121

In [19]:
def get_lda_topics(model, num_topics):
    """Print lda topics with pd.DataFrame"""
    
    word_dict = {}
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10)
        word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in words]
        
    return pd.DataFrame(word_dict).transpose()

### A. Nouns only

In [20]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [21]:
# Apply the nouns function to the transcripts to filter only on nouns
speech_df['nouns'] = speech_df.transcript.apply(nouns)

In [22]:
generate_pyLDAvis(speech_df,'nouns',5)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.120024  0.055070       1        1  27.203500
4      0.068605 -0.003772       2        1  27.029098
0      0.023479  0.072771       3        1  16.885199
1     -0.064132 -0.106164       4        1  15.258667
2      0.092072 -0.017905       5        1  13.623536, topic_info=          Term        Freq       Total Category  logprob  loglift
677      dream  615.000000  615.000000  Default  30.0000  30.0000
420    company  389.000000  389.000000  Default  29.0000  29.0000
264   business  400.000000  400.000000  Default  28.0000  28.0000
2238     women  663.000000  663.000000  Default  27.0000  27.0000
1266      love  444.000000  444.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
804    failure   54.434717  245.352888   Topic5  -5.7459   0.4877
973        guy   56.438792  314.895871   Topic5  -5.7097   0.2743
981       hand   58.856334  413.162833   Topic5  -5.6678   0.0446
1946     state   60.233292  569.519885   Topic5  -5.6446  -0.2532
1404     night   54.262933  340.762707   Topic5  -5.7490   0.1560

[378 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5         1  0.091773  accent
5         2  0.917734  accent
8         1  0.196449  access
8         3  0.152794  access
8         4  0.633003  access
...     ...       ...     ...
2272      1  0.087529   youve
2272      2  0.402634   youve
2272      3  0.179435   youve
2272      4  0.078776   youve
2272      5  0.253834   youve

[1064 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 2, 3])

### B. Nouns and adjective

In [23]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [24]:
# Apply the nouns function to the transcripts to filter only on nouns
speech_df['nouns_adj'] = speech_df.transcript.apply(nouns_adj)
speech_df.head()

,speaker,year,transcript,length,nouns,nouns_adj
0,SIDDHARTHA MUKHERJEE,2018,i wish someone have tell me at my own commence...,14487,i someone commencement requirement graduation ...,i someone own commencement requirement graduat...
1,ABBY WAMBACK,2018,failure be not something to be ashamed of its ...,15866,failure something something power failure octa...,failure something something power failure high...
2,JON B. FISHER,2018,thank you very much my father commencement spe...,8544,thank father commencement speaker martin luthe...,thank much father commencement speaker great m...
3,MINDY KALING,2018,good morning to the class of the faculty the p...,15391,morning class faculty parent grandparents hono...,good morning class faculty parent grandparents...
4,JESMYN WARD,2018,persist be patient be well good morning it be ...,14063,persist morning honor pleasure share day presi...,persist patient good morning honor pleasure sh...


In [25]:
generate_pyLDAvis(speech_df,'nouns_adj',5)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.042027  0.033712       1        1  29.602058
0      0.083107 -0.133678       2        1  22.839035
3     -0.072163 -0.016204       3        1  21.397480
4     -0.083473  0.009851       4        1  17.854039
1      0.114557  0.106318       5        1   8.307388, topic_info=         Term        Freq       Total Category  logprob  loglift
525   company  389.000000  389.000000  Default  30.0000  30.0000
3012    women  664.000000  664.000000  Default  29.0000  29.0000
2383  science  236.000000  236.000000  Default  28.0000  28.0000
148       art  198.000000  198.000000  Default  27.0000  27.0000
3014   wonder  144.000000  144.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1812  morning   50.658112  291.433583   Topic5  -5.6043   0.7383
2511    small   51.807001  342.748797   Topic5  -5.5819   0.5986
2100    power   52.729028  375.193148   Topic5  -5.5642   0.5258
1600      law   42.182098  272.757447   Topic5  -5.7874   0.6214
1059   father   41.167042  316.317937   Topic5  -5.8118   0.4489

[391 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         1  0.103309  absence
4         3  0.154963  absence
4         4  0.154963  absence
4         5  0.568199  absence
32        4  0.991042    actor
...     ...       ...      ...
3060      1  0.474690    youve
3060      2  0.040115    youve
3060      3  0.344317    youve
3060      4  0.120344    youve
3060      5  0.020057    youve

[1111 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 2])